# Evaluation Retrival

Um ganze sätze zu prüfen und zu vergleichen wird ebenfalls ein Cosinus-Similarity genutzt. Hierzu werden die Sätze in Vektoren umgewandelt und dann verglichen.

Die Testdaten wurden von Claude Code anhand der Produktdatei generiert. Sinnvoll wäre es, mit dem Kunden zu sprechen und echte Fragen zu sammeln.

prdukt_chunks muss so ;)

In [20]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from tqdm.notebook import tqdm
import json
import chromadb

client = chromadb.PersistentClient(path="../3-indexing/chroma_db")
collection = client.get_collection('prdukt_chunks')
model = SentenceTransformer('deepset/gbert-large')

No sentence-transformers model found with name deepset/gbert-large. Creating a new one with mean pooling.


In [21]:
temp_queries = [
      "Temperaturalarm bei Störung",           # Direkt aus Specs
      "Warnung bei Temperaturabweichung",
      "Optischer und akustischer Alarm",       # Aus Specs
      "SmartMonitoring Benachrichtigung",      # Aus Description  
      "E-Mail Warnung bei Temperaturproblem"   # Aus Description
  ]

size_queries = [
    "Außenmaße des Geräts",                  # Direkt aus Specs
    "Wie groß ist der Kühlschrank?",
    "Abmessungen in Zentimetern",            # Specs: "59,7 x 65,4 x 188,4" 
    "BxTxH Maße",
    "Platzbedarf des Medikamentenkühlschranks"
]

connectivity_queries = [
    "USB-Schnittstelle zum Datenauslesen",   # Direkt aus Description
    "WiFi LAN Verbindung",                   # Aus Specs: "LAN / WiFi integriert"
    "Netzwerk-Anschluss verfügbar",
    "Datenlogger per USB auslesen",          # Aus Specs
    "SmartModule Vernetzbarkeit"             # Aus Specs
]

energy_queries = [
    "Stromverbrauch pro Jahr",               # Specs: "172 kWh"
    "Energieverbrauch in 365 Tagen",         # Direkt aus Specs
    "Wie viel kWh verbraucht das Gerät?",
    "Betriebskosten Strom",
    "Ultra-efficient Energieeffizienz"      # Aus Description
]

safety_queries = [
    "Was passiert bei Stromausfall?",        # Description hat ganzen Abschnitt
    "Netzausfallalarm verfügbar",           # Specs: "unmittelbar bei Netzunterbrechung"
    "Batteriegepufferter Alarm",            # Description
    "SafetyDevice Schutzfunktion",          # Description
    "SmartLock automatische Verriegelung"   # Description
]

capacity_queries = [
    "Rauminhalt des Kühlschranks",          # Specs: "394 / 235 l"
    "Brutto Netto Liter",                   # Direkt aus Specs  
    "Wie viel passt in den Kühlschrank?",
    "Volumen für Medikamente",
    "Anzahl Schubfächer verfügbar"         # Specs: "8"
]

all_queries = temp_queries + size_queries + connectivity_queries + energy_queries + safety_queries + capacity_queries

## Retrieval

In [22]:
def retrieve(queries, top_k=10):
    
    queries_embedded = model.encode(queries, normalize_embeddings=True, show_progress_bar=True)
    results = []
    
    for i, query in enumerate(queries):

        result = collection.query(
            query_embeddings=[queries_embedded[i].tolist()], 
            n_results=top_k
        )

        for rank, (doc, dist) in enumerate(
            zip(
                result['documents'][0], 
                result['distances'][0]
            ), 
            start=1
        ):
            results.append({
                'query': query,
                'rank': rank,
                'document': doc,
                'distance': dist
            })

    return pd.DataFrame(results)

In [23]:
df = retrieve(all_queries)

Batches: 100%|██████████| 1/1 [00:01<00:00,  1.91s/it]


## Metriken

### Distance Metrics

Zeigt wie gut die Retivals sind.

L2-Distanz (Euklidean Distance) zwischen Query und Retrival Chunk im normalisierten Vektorspace. Je kleiner desto besser (0 = identisch, ~0,2 = sehr ähnlich, ...). 

- Ranking: Statistiken pro Rang
- Distance Gap: Abstand zwischen ersten und zehnten Rang
- Strong Results: Erbenisse mit einer Similarity > 0,3

In [ ]:
distance_stats = df.groupby('rank')['distance'].agg(['mean','median','std','min','max'])
print("Ranking")
print(distance_stats)
print("\n")

rank01_mean = df[df['rank'] == 1]['distance'].mean()
rank10_mean = df[df['rank'] == 10]['distance'].mean()
gap = rank10_mean - rank01_mean
print(f"Distance Gap (Rank 1 -> 10): {gap:.4f}")

# Cousin Sim
high_confidence = (df['distance'] < 0.3).sum()
total = len(df)
print(f"Strong Results: {high_confidence}/{total} ({high_confidence/total*100:.1f}%)")


          mean    median       std       min       max
rank                                                  
1     0.176155  0.168874  0.049239  0.076330  0.260699
2     0.180702  0.170687  0.050037  0.077264  0.260901
3     0.184541  0.179527  0.049604  0.084897  0.261605
4     0.187164  0.185214  0.049029  0.086334  0.262764
5     0.189703  0.188590  0.050367  0.089576  0.279184
6     0.192098  0.189495  0.050879  0.101294  0.294403
7     0.193670  0.191148  0.051310  0.103726  0.303758
8     0.194703  0.192510  0.051540  0.104139  0.306031
9     0.195652  0.193658  0.051526  0.104514  0.306102
10    0.196545  0.194766  0.051632  0.105209  0.309042


Distance Gap: 0.0204
High Confidence Result: 296/300 (98.7%)


### Consistency & Stability

- Tied Sum: Anzahl der Results mit gleicher Distanz -> Führtdazu, dass das Model nicht gut unterscheiden kann.

In [24]:
duplicate_distances = df.groupby(['query', 'distance']).size()
ties = (duplicate_distances > 1).sum()
print(f"Tied Sum: {ties}")

Tied Sum: 0


### Coverage

Anzahl der unterschiedlichen Chunks über alle Queries -> Zeigt die Abdeckung der Daten in der DB - oder ob immer die selben Chunks gefunden werden.

- Unique Chunks: Anzahl der Chunks sie einmalig gefunden wurden
- Most Wanted: Eigentlich die am häufigsten gefundenen Chunks

In [ ]:
unique_chunks = df['document'].nunique()
total_chunks_in_db = collection.count()
coverage = unique_chunks / total_chunks_in_db * 100
print(f"Unique Chunks: {unique_chunks}/{total_chunks_in_db} ({coverage:.1f}%)")
print("\n")

chunk_frequency = df['document'].value_counts()
most_common = chunk_frequency.head(5)
print("Most Wanted")
for doc, count in most_common.items():
    print(f"{count}x: {doc[:100]}...")

Unique Chunks: 197/1800 (10.9%)


7x: Sonstiges - flexible Ausstattung: True...
6x: Sonstiges - Netzkabel: 300, Zubehör: Edelstahlregale TT90-RC für bis zu 72 Standard-Cryoboxen, Optio...
6x: Sonstiges - Netzkabel: 300, Zubehör: Edelstahlregale TT90-RC, Optionen: vier Rollen fest montiert, z...
6x: Sonstiges - flexible Ausstattung möglich: True...
5x: Sonstiges - Netzkabel: 300, Zubehör: Edelstahlregale TT90-RC, Wifi-Bridge, Optionen: Rollen, Umluftv...
